In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
%matplotlib inline

In [31]:
features = ["OTHERS","WALKING","RUNNING","STANDING","SITTING","UPSTAIRS","DOWNSTAIRS"]
total_rawdata = pd.DataFrame()
feature_vec = ["label",
"linear_x_mean","linear_y_mean","linear_z_mean",
"gyro_x_mean","gyro_y_mean","gyro_z_mean",
"grav_x_mean","grav_y_mean","grav_z_mean",

"linear_x_std","linear_y_std","linear_z_std",
"gyro_x_std","gyro_y_std","gyro_z_std",
"grav_x_std","grav_y_std","grav_z_std",

"linear_x_energy","linear_y_energy","linear_z_energy",
"gyro_x_energy","gyro_y_energy","gyro_z_energy",
"grav_x_energy","grav_y_energy","grav_z_energy"
]
feature_df = pd.DataFrame(columns=feature_vec)
error = 0
window_num = 0
for i in range(7):
    count = 0
   
    
    raw_linear = pd.read_csv("20170252/" + str(i) +"/linear.csv", header=None )
    raw_linear.pop(raw_linear.columns[1])
   
    raw_gyro = pd.read_csv("20170252/" + str(i) +"/gyro.csv", header=None )
    raw_gyro.pop(raw_gyro.columns[0])
    raw_gyro.pop(raw_gyro.columns[0])
    raw_grav = pd.read_csv("20170252/" + str(i) +"/gravity.csv", header=None )
    raw_grav.pop(raw_grav.columns[0])
    raw_grav.pop(raw_grav.columns[0])


    merged_df = pd.concat([raw_grav, raw_linear], axis= 1)
    merged_df = pd.concat([merged_df, raw_gyro], axis= 1)
    merged_df.columns = ["label","linear_x","linear_y","linear_z","gyro_x","gyro_y","gyro_z","grav_x","grav_y","grav_z"]
    merged_df=merged_df.dropna()
    #print(merged_df.columns)
    
    merged_df['label'] = merged_df['label'].astype(int)
    #merged_df.to_csv(str(i)+ ".txt",index=None,header=None)
    #print(merged_df.dtypes)
    while True:
       
        if count + 200> merged_df.shape[0]:
            break
        window_num = window_num + 1
        window_df = merged_df[count : count + 200]
        window_df.pop(window_df.columns[0])
        if window_df.shape[0] != 200:
            error = error + 1
        
        ### calculation for the feature vector
        ### label, gravity xyz, linear xyz, gyro xyz
        ### mean, std, energy
        result=pd.concat([window_df.mean().to_frame().T,  window_df.std().to_frame().T,  window_df.pow(2).mean().to_frame().T],axis=1)
        result.insert(0,"label",i) 
        result.columns = feature_vec
        feature_df = pd.concat([feature_df,result])
        count = count + 100
#print(feature_df.shape)
feature_df.to_csv("feature.csv", index= False)
#print(window_num)

   








In [44]:
#to libsvm
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix

feature_df = pd.read_csv("feature.csv")
X = feature_df.drop("label", axis = 1)
y = feature_df["label"]

train_X, test_X, train_y, test_y = train_test_split(X,y,test_size = 0.2, random_state = 0)

svm = SVC()

svm.fit(train_X,train_y)
pred_y = svm.predict(test_X)

confmat = confusion_matrix(test_y,pred_y)
classrep = classification_report(test_y,pred_y,output_dict=True)

cdf = pd.DataFrame(confmat)
clrepdf =pd.DataFrame(classrep).transpose()



cdf.to_csv("confusion_matrix.csv")
clrepdf.to_csv("classification_report.csv")


print("\nConfusion matrix:\n others,walking,running,standing,sitting,upstairs,downstairs\n\n",confmat)
print("\nReport : \n",classrep)




Confusion matrix:
 others,walking,running,standing,sitting,upstairs,downstairs

 [[117   0   0   4   1   0   0]
 [  0  61   0   0   0   0   0]
 [  0   0  60   0   0   0   0]
 [  1   0   0  96   0   0   0]
 [  0   0   0   0  49   0   0]
 [  0   0   0   0   0  52   0]
 [  0   0   0   0   0   0  53]]

Report : 
 {'0': {'precision': 0.9915254237288136, 'recall': 0.9590163934426229, 'f1-score': 0.9749999999999999, 'support': 122}, '1': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 61}, '2': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 60}, '3': {'precision': 0.96, 'recall': 0.9896907216494846, 'f1-score': 0.9746192893401014, 'support': 97}, '4': {'precision': 0.98, 'recall': 1.0, 'f1-score': 0.98989898989899, 'support': 49}, '5': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 52}, '6': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 53}, 'accuracy': 0.9878542510121457, 'macro avg': {'precision': 0.9902179176755448, 'recall': 0.9